In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

import os
import joblib
import pickle

import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
import matplotlib.image as mpimg

import stability as st

import statistics
import scipy as scp
import math

import lime
from lime import lime_tabular

import shap

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/Breast Cancer/"
PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/Breast Cancer/"
model_filename = os.path.join(PATH, "models/diabetes_xgb.joblib")
dataset_path = 'datasets/'
model = joblib.load(model_filename)

In [4]:
tn_file = os.path.join(PATH, "datasets/dicts/true_neg.pkl")
with open (tn_file, 'rb') as f:
    results = pickle.load(f)
    
f.close()

In [9]:
X_train = pd.read_csv(PATH + "datasets/breast_cancer_Xtrain.csv", index_col=False).values
X_test = pd.read_csv(PATH + "datasets/breast_cancer_Xtest.csv", index_col=False).values
X_validation =pd.read_csv(PATH + "datasets/breast_cancer_Xvalidation.csv",index_col=False).values
y_train = pd.read_csv(PATH + "datasets/breast_cancer_Ytrain.csv",index_col=False).values
y_test =pd.read_csv(PATH + "datasets/breast_cancer_Ytest.csv", index_col=False).values
y_validation = pd.read_csv(PATH + "datasets/breast_cancer_Yvalidation.csv", index_col=False).values

original_data = pd.read_csv(PATH + "datasets/breast_cancer.csv")
original_data = original_data.dropna(axis = 'columns')
original_data = original_data.drop(['id'], axis = 1)
feature_names = original_data.drop(["diagnosis"], axis = 1).columns

In [12]:
feature_names

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [13]:
lime_exp.as_list()

[('0.25 < perimeter_worst <= 0.43', 0.14316127846613486),
 ('0.26 < radius_worst <= 0.44', 0.10729484216604683),
 ('0.11 < concave points_mean <= 0.24', -0.061378762296231786),
 ('0.14 < area_worst <= 0.27', 0.05763446995525868),
 ('texture_worst > 0.48', 0.041882108164151874),
 ('0.24 < concave points_worst <= 0.44', -0.03681699247233719),
 ('0.02 < area_se <= 0.04', -0.027860741677483286),
 ('0.08 < concavity_mean <= 0.20', -0.024568748965203523),
 ('texture_mean > 0.41', 0.016375399158249072),
 ('smoothness_worst <= 0.27', -0.014661198572049046),
 ('0.30 < radius_mean <= 0.48', 0.011818003483181195),
 ('0.09 < fractal_dimension_se <= 0.14', -0.011159548957340091),
 ('0.32 < symmetry_mean <= 0.41', 0.011113263171589494),
 ('smoothness_mean <= 0.38', -0.005731582482955103),
 ('0.11 < concavity_worst <= 0.22', -0.005163647421516782),
 ('0.10 < symmetry_se <= 0.15', -0.005093267219849103),
 ('fractal_dimension_mean <= 0.16', 0.004856006533981197),
 ('0.19 < symmetry_worst <= 0.25', -0.00466993545959343),
 ('0.05 < perimeter_se <= 0.08', -0.004492854728330131),
 ('0.29 < concave points_se <= 0.38', -0.00435258628924242),
 ('0.18 < area_mean <= 0.33', 0.004247577485821349),
 ('fractal_dimension_worst <= 0.11', 0.0030922248147330442),
 ('0.25 < texture_se <= 0.34', -0.0021997277759748858),
 ('0.06 < concavity_se <= 0.09', 0.0021662119164746886),
 ('0.15 < compactness_mean <= 0.27', -0.002155479114208586),
 ('0.31 < perimeter_mean <= 0.48', -0.0014661587313188264),
 ('compactness_worst <= 0.12', -0.0013377552845956436),
 ('0.05 < radius_se <= 0.09', -0.0004232796723333605),
 ('compactness_se <= 0.09', 0.0003002777642926255),
 ('smoothness_se <= 0.09', 0.0001525579072830892)]

In [10]:
MAX_FEAT = 10

start_time = time.time()

# LIME has one explainer for all the models
explainer = lime_tabular.LimeTabularExplainer(X_train, feature_names= feature_names, 
                                                  class_names=["Benign", "Malignant"], verbose=False, mode='classification')

elapsed = time.time() - start_time

print ("Time taken to create explainer:", round(elapsed, 2), "seconds")

Time taken to create explainer: 0.02 seconds


In [14]:
stability_scores = []
probas = []

feat_list = list(feature_names)

for instance in results:
    print ("Running instance", results.index(instance))
    
    feat_pres = []

    print("INDEX = %d" %instance['index'])
    patients_feat = np.array(instance['scaled_vector'])
    prediction = instance['predictions']
    probas.append(instance['prediction_probability'][0])
    
    start_time = time.time()
    # explain instance
    
    for iteration in range(100):
        print("Run", iteration)
        lime_exp = explainer.explain_instance(patients_feat, model.predict_proba, num_features = 30)
        
        #For stability by index
        presence_list = [0]*len(feat_list)
    
        for each in feat_list:
            list_idx = feat_list.index(each)
            #print ("Feature", list_idx)
            for explanation in lime_exp.as_list():
                if each in explanation[0]:
                    if lime_exp.as_list().index(explanation) < MAX_FEAT:
                        presence_list[list_idx] = 1
                        
        feat_pres.append(presence_list)
        
        #method 1 - Nogueira, Sechidis, Brown.
    stability = st.getStability(feat_pres[:10])
    print ("Stability:", round(stability,2))
    stability_scores.append(stability)
 
    print("\n--------------------------------------------------------------------------------------------\n")

Running instance 0
INDEX = 52
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78

--------------------------------------------------------------------------------------------

Running instance 1
INDEX = 5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.89

--------------------------------------------------------------------------------------------

Running instance 2
INDEX = 38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.73

--------------------------------------------------------------------------------------------

Running instance 3
INDEX = 21
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81

--------------------------------------------------------------------------------------------

Running instance 4
INDEX = 12
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.74

------------------------------------------------------------------------------------------

# SHAP

In [ ]:
X_train_frame = pd.DataFrame(data = X_train, columns = feature_names)
X_test_frame = pd.DataFrame(data = X_test, columns = feature_names)

data_sample = shap.kmeans(X_train_frame, 100)
shap_explainer = shap.KernelExplainer(model.predict, data_sample)#, link = 'identity')

shap.initjs()

In [ ]:
def create_samples(iterations, row, features, top = 10):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(X_test_frame.loc[row])

        importances = []

        for i in range(length):
            feat = features[i]
            shap_val = shap_values[i]
            abs_val = abs(shap_values[i])
            entry = (feat, shap_val, abs_val)
            importances.append(entry)
            #print(entry)

        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        for i in range(top):
            feat = importances[i]
            if feat[2] > 0:
                rel_feat.append(feat)
                
        rel_exp.append(rel_feat)
        
    return exp, rel_exp

In [ ]:
shap_stability_scores = []
probas = []

for instance in results:
    
    print("INDEX = %d" %instance['index'])
    patients_feat = pd.Series(instance['scaled_vector'])
    prediction = instance['predictions']
    probas.append(instance['prediction_probability'][0])
    
    loc = results.index(instance)
    
    exp, rel_exp = create_samples(100, loc, feature_names, top = 5)
    
    #print(rel_exp)
    
    feat_pres = []
    
    for iteration in rel_exp:
        print("Iteration", rel_exp.index(iteration))
        #print(iteration)

        #Stability by index
        presence_list = [0]*len(feat_list)
        
        for each in feat_list:
            list_idx = feat_list.index(each)
            #print ("Feature", list_idx)
            for explanation in iteration:
                if each in explanation[0]:
                    #by index
                    presence_list[list_idx] = 1
                    
        feat_pres.append(presence_list)
        
    stability = st.getStability(feat_pres)
    print ("Stability:", round(stability,2))
    shap_stability_scores.append(stability)

In [ ]:
plt.plot(probas, stability_scores, 'bo', label = 'LIME')
plt.plot(probas, shap_stability_scores, 'ro', label = 'SHAP')
plt.show()